In [ ]:
# assign data URL
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQzBYWDif8AqH47QpdaMsxZ0d3aXafgvL6EfnsUk6iN5QPCgrhvEky7hzI16iyfL3L2rfec3QX32JQj/pub?gid=0&single=true&output=csv'
data_url

In [27]:
import pandas
import numpy as np

In [ ]:
data = pandas.read_csv(data_url)

# Function to convert values with $ to integers
def convert_currency_to_int(value):
    if isinstance(value, str) and '$' in value:
        cleaned_value = value.replace("$","").replace(",","").strip()
        try:
            return int(cleaned_value)
        except ValueError:
            return value  # Handle the case where the value can't be converted to an integer
    else:
        return value

# Apply the conversion function to all elements in the DataFrame
data = data.applymap(convert_currency_to_int)

#display data
data

#display column names
data.columns

#list data types
data.info()

#generate descriptive stats for data
data.describe()

In [29]:
# Setting the noise/redundant/missing data variables
ignored_cols  = ['DonorUniqueId', 'PreferredAddressType', 'DonorPostalCode', 'DonorDateOfBirth', 'CumulativeDonationAmount']

In [30]:
# Setting categorical cols
cat_cols = ['GenderIdentity', 'IsMemberFlag', 'IsAlumnusFlag', 'IsParentFlag', 'HasInvolvementFlag', 'HasEmailFlag', 'MaritalStatus', 'WealthRating', 'AcademicDegreeLevel']

In [31]:
## Setting the numerical cols
num_cols = ['DonorAge', 'ConsecutiveDonorYears',
       'LastFiscalYearDonation', 'Donation2FiscalYearsAgo', 'Donation3FiscalYearsAgo',
       'Donation4FiscalYearsAgo', 'Donation5FiscalYearsAgo', 'CurrentFiscalYearDonation']

In [38]:
# Setting the dependent/target/label variable
y = 'DonorIndicatorFlag.'

In [ ]:
# Installing Pycaret for Automated ML
!pip install -U --pre pycaret

In [ ]:
# Installing sweetviz package for visualization and statistical analysis
!pip install sweetviz

In [13]:
# Importing the package
import sweetviz as sv

In [14]:
# Creating a variables with predictor columns
pred_cols  = ['DonorAge', 'ConsecutiveDonorYears',
       'LastFiscalYearDonation', 'Donation2FiscalYearsAgo', 'Donation3FiscalYearsAgo',
       'Donation4FiscalYearsAgo', 'Donation5FiscalYearsAgo', 'CurrentFiscalYearDonation',
       'GenderIdentity', 'IsMemberFlag', 'IsAlumnusFlag', 'IsParentFlag', 'HasInvolvementFlag',
       'HasEmailFlag', 'MaritalStatus', 'WealthRating', 'AcademicDegreeLevel', 'DonorIndicatorFlag.',
       'CumulativeDonationAmount']

In [ ]:
# Generating a dataset with only predictor columns
pred_data = data[pred_cols]

# Displaying the predictors dataset
pred_data

In [ ]:
# Generating EDA report using sweetviz
sv_report = sv.analyze(pred_data)

In [ ]:
# Generate a html formatted report
sv_report.show_html("Red Cross Report.html")

In [ ]:
# Installing dataprep package for data profiling
!pip install dataprep

# Importing dataprep package for data profiling
from dataprep.eda import create_report

# Generating the data profiling report
create_report(data).save()

In [ ]:
# Importing the pycaret and setting the experiment
from pycaret.classification import *

classification_model_setup = setup(data,
                                   target  = y,
                                   ignore_features=ignored_cols,
                                   numeric_features=num_cols,
                                   categorical_features=cat_cols,
                                   train_size = 0.7)

In [39]:
# Invoking ML algorithms
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6970,0.7569,0.6970,0.8210,0.6927,0.4402,0.5223,2.2110
ada,Ada Boost Classifier,0.6960,0.7553,0.6960,0.8185,0.6918,0.4379,0.5187,1.0340
lr,Logistic Regression,0.6915,0.7572,0.6915,0.7812,0.6911,0.4187,0.4734,1.8860
lightgbm,Light Gradient Boosting Machine,0.6890,0.7562,0.6890,0.7687,0.6898,0.4099,0.4566,1.4120
xgboost,Extreme Gradient Boosting,0.6887,0.7561,0.6887,0.7638,0.6899,0.4071,0.4503,0.7430
qda,Quadratic Discriminant Analysis,0.6856,0.7549,0.6856,0.8234,0.6790,0.4237,0.5144,0.7090
dt,Decision Tree Classifier,0.6829,0.7374,0.6829,0.7299,0.6868,0.3799,0.4024,0.4850
rf,Random Forest Classifier,0.6811,0.7554,0.6811,0.7202,0.6855,0.3705,0.3875,2.1710
et,Extra Trees Classifier,0.6773,0.7348,0.6773,0.7247,0.6812,0.3693,0.3915,2.9170
svm,SVM - Linear Kernel,0.6583,0.0000,0.6583,0.6511,0.5946,0.2313,0.2753,1.1630


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=4418, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
#Using the best performing model
model = create_model('gbc')

In [34]:
# Importing the client dataset
client_data = pandas.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQfRxnJl2drvo5fEUs8toEpIiniUlvr91eVYumIkCOKTewYMQ8f2WFY6XzKZyad51fOPZpxPcu9AOhf/pub?gid=420791075&single=true&output=csv')
client_data.shape

(6270, 22)

In [35]:
# Predicting standard scores using the best model
client_data = client_data.applymap(convert_currency_to_int)
client_data_predictions = predict_model(model, client_data)

In [ ]:
client_data_predictions

In [37]:
client_data_predictions.to_csv("Red Cross -ML Algo.csv")